In [1]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('../')
from mrcnn.utils import trim_zeros_graph
import numpy as np
import tensorflow as tf

np.set_printoptions(linewidth=100, precision=4)
sess = tf.InteractiveSession()

batch_size = 3
num_rois = 32
num_classes = 4
num_detections = 100
num_rois_per_image = 32
rois_per_image = 32

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# def build_gaussian_tf(in_tensor) :   #, pred_cls_cnt, config):

num_detections  = 32          # config.DETECTION_MAX_INSTANCES
h, w            = [128, 128]  #   config.IMAGE_SHAPE[:2]
num_cols        = 7 
img_h, img_w    = [128, 128]  # config.IMAGE_SHAPE[:2]
batch_size      = 3           # config.BATCH_SIZE
num_classes     = 4           # config.NUM_CLASSES  
print('>>> build_gaussian_tf ')

## rois per image is determined by size of input tensor 
##   detection mode:   config.TRAIN_ROIS_PER_IMAGE 
##   ground_truth  :   config.DETECTION_MAX_INSTANCES    

##-----------------------------------------------------------------------------
## Build mesh-grid to hold pixel coordinates 
##-----------------------------------------------------------------------------
X = tf.range(img_w, dtype=tf.int32)
Y = tf.range(img_h, dtype=tf.int32)
X, Y = tf.meshgrid(X, Y)
# grid1 = tf.stack([X,Y], axis=-1)
# print('grid1 shape ', grid1.shape)
# print(grid1[0,:,:].eval())

print('   -- build meshgrid -----')
print('   X and Y meshgrid shapes ', X.get_shape(), Y.get_shape())
# print( ' X : \n',X.eval())
# print( ' Y : \n',Y.eval())

# ## hear we repeat X and Y  batch_size x rois_per_image times
ones = tf.ones([batch_size, rois_per_image,1, 1], dtype = tf.int32)
print('   ones: ',ones.shape)                
# # ones = tf.expand_dims(ones,-1)
# print(' ones with exp dims ',ones.shape)

rep_X = ones * X
rep_Y = ones * Y 
# print(' ones_exp * X', ones.shape, '*', X.shape, '= ',rep_X.shape)
# print(' ones_exp * Y', ones.shape, '*', Y.shape, '= ',rep_Y.shape)

# # stack the X and Y grids 
bef_pos = tf.to_float(tf.stack([rep_X,rep_Y], axis = -1))
print(' before transpse ', bef_pos.get_shape())
pos_grid = tf.transpose(bef_pos,[2,3,0,1,4])
print('   after transpose ', pos_grid_1.get_shape())    
print(pos_grid[:,:,0,0,:].eval())

>>> build_gaussian_tf 
   -- build meshgrid -----
   X and Y meshgrid shapes  (128, 128) (128, 128)
   ones:  (3, 32, 1, 1)
 before transpse  (3, 32, 128, 128, 2)
   after transpose  (128, 128, 3, 32, 2)
[[[  0.   0.]
  [  1.   0.]
  [  2.   0.]
  ...
  [125.   0.]
  [126.   0.]
  [127.   0.]]

 [[  0.   1.]
  [  1.   1.]
  [  2.   1.]
  ...
  [125.   1.]
  [126.   1.]
  [127.   1.]]

 [[  0.   2.]
  [  1.   2.]
  [  2.   2.]
  ...
  [125.   2.]
  [126.   2.]
  [127.   2.]]

 ...

 [[  0. 125.]
  [  1. 125.]
  [  2. 125.]
  ...
  [125. 125.]
  [126. 125.]
  [127. 125.]]

 [[  0. 126.]
  [  1. 126.]
  [  2. 126.]
  ...
  [125. 126.]
  [126. 126.]
  [127. 126.]]

 [[  0. 127.]
  [  1. 127.]
  [  2. 127.]
  ...
  [125. 127.]
  [126. 127.]
  [127. 127.]]]


In [6]:
tfd        = tf.contrib.distributions
grid       = pos_grid[:,:,0,0,:]
covar      = np.array([27.7818, 26.6678],dtype = np.float32)
covar_sqrt = np.sqrt(covar)
covar_sqrd = covar ** 2
full_covar = np.array([[27.7818, 0],[0, 26.6678]],dtype = np.float32)
mean       = np.array([48.8926, 36.101 ],dtype = np.float32)

print('   grid :', grid.dtype, grid.shape)
print('   Covar sqrt :', covar_sqrt)
print('   Covar sqrd :', covar_sqrd)

mvn1  = tfd.MultivariateNormalDiag(loc=mean,scale_diag=covar_sqrt)
prob1 = mvn1.prob(grid)
print()
print('tfd.MultivariateNormalDiag')
print('   mvn1 mean             ', mvn1.mean().eval())
print('   mvn1 std deviation    ', mvn1.stddev().eval())
print('   mvn1 covariance:      ', '\n', mvn1.covariance().eval())
print('   mvn1 location         ', mvn1.loc.eval())
print('   Linear OP shape       ', mvn1.scale.shape)
print('   Linear Op batch shape ', mvn1.scale.batch_shape)
print('   Linear op Range Dim   ', mvn1.scale.range_dimension)
print('   Linear op Domain Dim  ', mvn1.scale.domain_dimension) 
print('   Linear op Domain Dim  ', mvn1.scale.diag_part().eval()) 


mvn2  = tfd.MultivariateNormalDiag(loc=mean,scale_diag=covar)
prob2 = mvn2.prob(grid)
print()
print('tfd.MultivariateNormalDiag')
print('   mvn2 mean             ', mvn2.mean().eval())
print('   mvn2 std deviation    ', mvn2.stddev().eval())
print('   mvn2 covariance:      ', '\n', mvn2.covariance().eval())
print('   mvn2 location         ', mvn2.loc.eval())
print('   Linear OP shape       ', mvn2.scale.shape)
print('   Linear Op batch shape ', mvn2.scale.batch_shape)
print('   Linear op Range Dim   ', mvn2.scale.range_dimension)
print('   Linear op Domain Dim  ', mvn2.scale.domain_dimension) 
print('   Linear op Domain Dim  ', mvn2.scale.diag_part().eval()) 


mvn3  = tfd.MultivariateNormalFullCovariance( loc = mean, covariance_matrix = full_covar)
prob3 = mvn3.prob(grid)
print()
print('tfd.MultivariateNormalFullCovariance')
print('   mvn3 mean             ', mvn3.mean().eval())
print('   mvn3 std deviation    ', mvn3.stddev().eval())
print('   mvn3 covariance:      ', '\n', mvn3.covariance().eval())
print('   mvn3 location         ', mvn3.loc.eval())
print('   Linear OP shape       ', mvn3.scale.shape)
print('   Linear Op batch shape ', mvn3.scale.batch_shape)
print('   Linear op Range Dim   ', mvn3.scale.range_dimension)
print('   Linear op Domain Dim  ', mvn3.scale.domain_dimension) 
print('   Linear op Domain Dim  ', mvn3.scale.diag_part().eval()) 

print('   << output probabilities shape:' )
print(' prob1 ', prob1.get_shape())
print(prob1.eval())
print(' prob2 ', prob2.get_shape())
print(prob2.eval())
print(' prob3 ', prob3.get_shape())
print(prob3.eval())

   grid : <dtype: 'float32'> (128, 128, 2)
   Covar sqrt : [5.2708 5.1641]
   Covar sqrd : [771.8284 711.1716]

tfd.MultivariateNormalDiag
   mvn1 mean              [48.8926 36.101 ]
   mvn1 std deviation     [5.2708 5.1641]
   mvn1 covariance:       
 [[27.7818  0.    ]
 [ 0.     26.6678]]
   mvn1 location          [48.8926 36.101 ]
   Linear OP shape        (2, 2)
   Linear Op batch shape  ()
   Linear op Range Dim    2
   Linear op Domain Dim   2
   Linear op Domain Dim   [5.2708 5.1641]

tfd.MultivariateNormalDiag
   mvn2 mean              [48.8926 36.101 ]
   mvn2 std deviation     [27.7818 26.6678]
   mvn2 covariance:       
 [[771.8284   0.    ]
 [  0.     711.1716]]
   mvn2 location          [48.8926 36.101 ]
   Linear OP shape        (2, 2)
   Linear Op batch shape  ()
   Linear op Range Dim    2
   Linear op Domain Dim   2
   Linear op Domain Dim   [27.7818 26.6678]

tfd.MultivariateNormalFullCovariance
   mvn3 mean              [48.8926 36.101 ]
   mvn3 std deviation     [5.

In [7]:
np.set_printoptions(linewidth=150, threshold=10000)
from scipy.stats import  multivariate_normal
# Build mesh-grid to hold pixel coordinates ----------------------------------
XX = np.arange(0, img_w, 1)
YY = np.arange(0, img_h, 1)
XX, YY = np.meshgrid(XX, YY)
pos  = np.empty(XX.shape + (2,))   # concatinate shape of x to make ( x.rows, x.cols, 2)
pos[:,:,0] = XX;
pos[:,:,1] = YY;
# print(XX)
# print(YY)
# print(pos[0,:,:])
# print(pos[0])
# print(grid[0].eval())
print(' pos type    ', type(pos), type(grid))
print(' grid shape ', pos.shape, grid.shape)
print(np.all(pos == grid.eval()))
print(' mean  ', mean)
print(' covar ', covar)
mvna   = multivariate_normal(mean, covar)
prob_a = mvna.pdf(pos)

mvnb   = multivariate_normal(mean, covar_sqrd)
prob_b = mvnb.pdf(pos)

print(prob_a[35:50, 45:54])
max_a = np.max(prob_a)
print('Postiion of max_a in tensor prob_a is :', np.unravel_index(np.argmax(prob_a) , prob_a.shape) )

print()

print(' covar ', covar_sqrd)
print(prob_b[35:50, 45:54])
max_b = np.max(prob_b)
print('Postiion of max_b in tensor prob_b is :', np.unravel_index(np.argmax(prob_b) , prob_b.shape) )

print('max a , max_b ', max_a, max_b, max_a/max_b)

 pos type     <class 'numpy.ndarray'> <class 'tensorflow.python.framework.ops.Tensor'>
 grid shape  (128, 128, 2) (128, 128, 2)
True
 mean   [48.8926 36.101 ]
 covar  [27.7818 26.6678]
[[0.0044 0.0049 0.0054 0.0056 0.0057 0.0056 0.0053 0.0048 0.0042]
 [0.0045 0.005  0.0055 0.0058 0.0058 0.0057 0.0054 0.0049 0.0043]
 [0.0044 0.005  0.0054 0.0057 0.0058 0.0056 0.0053 0.0048 0.0043]
 [0.0042 0.0047 0.0051 0.0054 0.0055 0.0053 0.005  0.0046 0.004 ]
 [0.0038 0.0043 0.0047 0.0049 0.005  0.0049 0.0046 0.0042 0.0037]
 [0.0033 0.0038 0.0041 0.0043 0.0044 0.0043 0.0041 0.0037 0.0032]
 [0.0028 0.0032 0.0035 0.0037 0.0037 0.0036 0.0034 0.0031 0.0028]
 [0.0023 0.0026 0.0029 0.003  0.003  0.003  0.0028 0.0026 0.0022]
 [0.0018 0.0021 0.0022 0.0024 0.0024 0.0023 0.0022 0.002  0.0018]
 [0.0014 0.0016 0.0017 0.0018 0.0018 0.0018 0.0017 0.0015 0.0013]
 [0.001  0.0011 0.0012 0.0013 0.0013 0.0013 0.0012 0.0011 0.001 ]
 [0.0007 0.0008 0.0009 0.0009 0.0009 0.0009 0.0009 0.0008 0.0007]
 [0.0005 0.0005 0.0006 